In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Anti-backlash test data

In [27]:
def average_test_repetitions(df, groupby):
    data = []
    grouped = df.groupby(groupby)

    for name, group in grouped:
        group = group.drop(groupby, axis=1)
        group.index = group.index - group.index[0]

        old_len = len(group)
        group = group.resample('0.5ms').mean()
        group = group.interpolate(method='time')
        # print(f'test {name} resampled from {old_len} to {len(group)} samples')

        data.append(group.values)

    # get consistent length
    min_len = min([len(d) for d in data])
    data = [d[:min_len] for d in data]
    # print(f'setting everything to {min_len} samples')

    # average data using numpy
    data_mean = np.mean(np.array(data), axis=0)

    new_columns = df.columns.drop(groupby)
    mean_df = pd.DataFrame(data_mean, columns=new_columns)

    # smooth data
    mean_df = mean_df.rolling(window=100, center=True).mean()
    mean_df = mean_df.bfill().ffill()

    return mean_df


class actuator_test_data:
    def __init__(self, configuration='...'):
        self.configuration = configuration
        self.torque_ramps = {}
        self.speed_ramps = {}
        self.runin_tests = []

        
    def add_test_moment(self, phase, files):
        torque_ramp_files = [f for f in files if 'torqueramp' in f]
        speed_ramp_files = [f for f in files if 'speedramp' in f]
        
        if torque_ramp_files:
            self.add_torque_ramps(phase, torque_ramp_files)
        
        for file in speed_ramp_files:
            self.add_speed_ramp(phase, file)

    def add_torque_ramps(self, name, files):
        torque_ramps = {}
        for filename in files:    
            print(f'prepping {filename}')
            df_raw = pd.read_csv(filename)

            df = pd.DataFrame()
            df['Time [s]'] = df_raw['TIME']
            df['Motor Torque [Nm]'] = df_raw['TORQUE']
            df['Deflection [deg]'] = df_raw['POSITION'] * 360
            df['Desired Torque [Nm]'] = df_raw['CONTROL_TORQUE']
            df['Q_current/5'] = df_raw['Q_CURRENT']/5
            df['test_nr'] = df_raw['test_nr']

            # print(f'Sample rate: {1/(df["Time [s]"].diff().mean())} Hz')
            df['Time [s]'] = pd.to_datetime(df['Time [s]'], unit='s')
            df.set_index('Time [s]', inplace=True)


            print(f"Test output orientation: {df['Deflection [deg]'].mean()} deg")
            df['Deflection [deg]'] -= df['Deflection [deg]'].mean()

            
            TEST_NR_OFFSET = 100
            play_tests = df[df['test_nr'] < TEST_NR_OFFSET].copy()
            stiffness_tests = df[df['test_nr'] >= TEST_NR_OFFSET].copy()
            stiffness_tests['test_nr'] = stiffness_tests['test_nr'] - TEST_NR_OFFSET

            play_test_mean = average_test_repetitions(play_tests, 'test_nr')
            stiffness_test_mean = average_test_repetitions(stiffness_tests, 'test_nr')

            torque_ramps[filename] = (play_test_mean, stiffness_test_mean)
        
        self.torque_ramps[name] = torque_ramps

    def add_speed_ramp(self, name, filename):
        print(f'prepping {filename}')
        df_raw = pd.read_csv(filename)

        df = pd.DataFrame()
        df['Motor Speed [rpm]'] = df_raw['VELOCITY']
        df['Motor Torque [Nm]'] = df_raw['TORQUE']

        df['Time [s]'] = pd.to_datetime(df_raw['TIME'], unit='s')
        df.set_index('Time [s]', inplace=True)  
        self.speed_ramps[name] = df

    def add_runin_tests(self, files):
        def read_file(filename):
            df_raw = pd.read_csv(filename, sep=';')
            df = pd.DataFrame()
            df['Time [s]'] = pd.to_datetime(df_raw['TIME'] / 1e9, unit='s')
            df['Time [s]'] = df['Time [s]'] - df['Time [s]'][0]
            df.set_index('Time [s]', inplace=True)

            if 'DIRECTION' not in df_raw.columns:
                df_raw['DIRECTION'] = 1
            df['Motor Torque [Nm]'] = df_raw['TORQUE'] * df_raw['DIRECTION']
            df['Averaged Torque [Nm]'] = df['Motor Torque [Nm]'].rolling(window=100, center=True).mean()
            df['Motor Temperature [c]'] = df_raw['MOTOR_TEMPERATURE'] *0.442 - 1.62   #approximation to compensate incorrect NTC
            df['Controller Temperature'] = df_raw['TEMPERATURE']
            
            return df

        df = read_file(files[0])
        df['test number'] = 0
        for filename in files[1:]:
            df2 = read_file(filename)
            df2.index = df2.index + df.index[-1] + pd.Timedelta(1, unit='ms')
            df2['test number'] = df['test number'].iloc[-1] + 1
            df = pd.concat([df, df2])

        self.runin_tests = df

    def describe(self):
        print(f'Configuration: {self.configuration}')
        print(f'Speed ramps: {list(self.speed_ramps.keys())}')
        print(f'Torque ramps:')
        for name, ramps in self.torque_ramps.items():
            print(f'  {name}: {list(ramps.keys())}')
        print(f'Run-in tests: {self.runin_tests["test number"].max()+1} tests, total time: {self.runin_tests.index[-1]}')

In [29]:
tests_split_pccf = actuator_test_data('Split Pinwheel (PC-CF)')
tests_split_pccf.add_test_moment('initial', [
    'test_data/2024-07-10__12-06-49_speedramp_split init_120s.csv',
    # 'test_data/2024-07-10__12-30-32__torqueramp__split init1.csv',
    'test_data/2024-07-10__12-34-04__torqueramp__split init1.csv',
    'test_data/2024-07-10__12-39-58__torqueramp__split init5.csv',
    'test_data/2024-07-10__12-45-53__torqueramp__split init6.csv', 
])
tests_split_pccf.add_runin_tests([
    'test_data/2024-07-10__15-56-23_run-in_split_0.35rps.csv',
])
tests_split_pccf.add_test_moment('after run-in', [
    'test_data/2024-07-10__16-10-16_speedramp_split 10min_120s.csv',
    'test_data/2024-07-10__16-27-11__torqueramp__split runin1.csv',
    'test_data/2024-07-10__16-31-56__torqueramp__split runin5.csv',
    'test_data/2024-07-10__16-35-37__torqueramp__split runin6.csv',
    # 'test_data/2024-07-10__16-39-51_speedramp_split runin_120s.csv',
])
tests_split_pccf.add_test_moment('preload increase 1', [
    'test_data/2024-07-10__16-44-11_speedramp_split 1pre_120s.csv',
    'test_data/2024-07-10__16-48-09__torqueramp__split 1pre1.csv',
    'test_data/2024-07-10__16-52-42__torqueramp__split 1pre5.csv',
    'test_data/2024-07-10__16-58-03__torqueramp__split 1pre6.csv',
])

    # 'test_data/2024-07-11__11-11-47_speedramp_split 2pre_120s.csv',
    # 'test_data/2024-07-11__11-17-34_speedramp_split 1.5pre_120s.csv',
tests_split_pccf.add_test_moment('preload increase 2', [
    'test_data/2024-07-11__11-26-10_speedramp_split 2pre_120s.csv',
    'test_data/2024-07-11__11-32-34__torqueramp__split 2pre1.csv',
    'test_data/2024-07-11__11-37-19__torqueramp__split 2pre5.csv',
    'test_data/2024-07-11__11-42-35__torqueramp__split 2pre6.csv',
])
tests_split_pccf.add_test_moment('preload increase 3', [
    'test_data/2024-07-11__12-18-00_speedramp_split 3pre_120s.csv',
    'test_data/2024-07-11__12-24-10__torqueramp__split 3pre1.csv',
    'test_data/2024-07-11__12-29-25__torqueramp__split 3pre5.csv',
    'test_data/2024-07-11__12-43-57__torqueramp__split 3pre6.csv',
])
tests_split_pccf.add_test_moment('preload increase 4', [
    'test_data/2024-07-11__13-20-23_speedramp_split 4pre_120s.csv',
    'test_data/2024-07-11__13-26-36__torqueramp__split 4pre1.csv',
    'test_data/2024-07-11__13-42-44__torqueramp__split 4pre5.csv',
    'test_data/2024-07-11__13-46-33__torqueramp__split 4pre6.csv',
])
tests_split_pccf.add_test_moment('preload increase 5', [
        'test_data/2024-07-11__13-51-30_speedramp_split 5pre_120s.csv',
])

print('\n')
tests_split_pccf.describe()

prepping test_data/2024-07-10__12-34-04__torqueramp__split init1.csv
Test output orientation: 11.163737800981444 deg
prepping test_data/2024-07-10__12-39-58__torqueramp__split init5.csv
Test output orientation: -2.9686886714726626 deg
prepping test_data/2024-07-10__12-45-53__torqueramp__split init6.csv
Test output orientation: 7.383565216576946 deg
prepping test_data/2024-07-10__12-06-49_speedramp_split init_120s.csv
prepping test_data/2024-07-10__16-27-11__torqueramp__split runin1.csv
Test output orientation: 3.719158996228808 deg
prepping test_data/2024-07-10__16-31-56__torqueramp__split runin5.csv
Test output orientation: -9.837408733440672 deg
prepping test_data/2024-07-10__16-35-37__torqueramp__split runin6.csv
Test output orientation: 0.26258394002733165 deg
prepping test_data/2024-07-10__16-10-16_speedramp_split 10min_120s.csv
prepping test_data/2024-07-10__16-48-09__torqueramp__split 1pre1.csv
Test output orientation: 3.9134693292870253 deg
prepping test_data/2024-07-10__16-52-

In [ ]:
tests_base_pccf = actuator_test_data('Baseline (PC-CF)')
tests_base_pccf.add_test_moment('initial', [
    'test_data/2024-07-11__14-38-26_speedramp_base init_120s.csv',
    # 'test_data/2024-07-11__14-44-21__torqueramp__base init1.csv',
    # 'test_data/2024-07-11__14-50-03__torqueramp__base init5.csv',         # locking plate not fixed correctly
    'test_data/2024-07-11__14-54-07__torqueramp__base init1.csv',
    'test_data/2024-07-11__14-59-51__torqueramp__base init5.csv',
    'test_data/2024-07-11__15-06-17__torqueramp__base init6.csv',
])
tests_base_pccf.add_runin_tests([
    'test_data/2024-07-11__15-12-31_run-in_base_0.35rps.csv',
])
tests_base_pccf.add_test_moment('after run-in', [
    'test_data/2024-07-11__15-26-00_speedramp_base runin_120s.csv',
    'test_data/2024-07-11__15-47-12__torqueramp__base runin1.csv',
    'test_data/2024-07-11__15-51-51__torqueramp__base runin5.csv',
    'test_data/2024-07-11__15-56-27__torqueramp__base runin6.csv',
])

In [ ]:
tests_conic_pccf = actuator_test_data('Conic Disk (PC-CF)')
tests_conic_pccf.add_test_moment('initial', [
    'test_data/2024-07-12__11-25-30_speedramp_conic init_120s.csv',
    'test_data/2024-07-12__11-33-31__torqueramp__conic init1.csv', 
    'test_data/2024-07-12__12-23-01__torqueramp__conic init5.csv',
    'test_data/2024-07-12__12-29-20__torqueramp__conic init6.csv',
])
tests_conic_pccf.add_runin_tests([
    'test_data/2024-07-12__12-40-29_run-in_conic_0.35rps.csv',
])
tests_conic_pccf.add_test_moment('after run-in', [
    'test_data/2024-07-12__12-57-37_speedramp_conic runin_120s.csv',
    'test_data/2024-07-12__13-05-47__torqueramp__conic runin1.csv',
    'test_data/2024-07-12__13-16-01__torqueramp__conic runin5.csv',
    'test_data/2024-07-12__13-32-22__torqueramp__conic runin6.csv',

])

# initial data visualization

In [ ]:
actuators = [tests_split_pccf, tests_base_pccf, tests_conic_pccf]

for actuator in actuators:

In [3]:

def average_test_repetitions(df, groupby):
    data = []
    grouped = df.groupby('groupby')

    for name, group in grouped:
        group = group.drop('groupby', axis=1)
        group.index = group.index - group.index[0]

        old_len = len(group)
        group = group.resample('0.5ms').mean()
        group = group.interpolate(method='time')
        # print(f'test {name} resampled from {old_len} to {len(group)} samples')

        data.append(group.values)

    # get consistent length
    min_len = min([len(d) for d in data])
    data = [d[:min_len] for d in data]
    # print(f'setting everything to {min_len} samples')

    # average data using numpy
    data_mean = np.mean(np.array(data), axis=0)

    new_columns = df.columns.drop('groupby')
    mean_df = pd.DataFrame(data_mean, columns=new_columns)

    # smooth data
    mean_df = mean_df.rolling(window=100, center=True).mean()
    mean_df = mean_df.bfill().ffill()

    return mean_df


def load_torquedata(filename):
    print(f'prepping {filename}')
    df_raw = pd.read_csv(filename)

    df = pd.DataFrame()
    df['Time [s]'] = df_raw['TIME']
    df['Motor Torque [Nm]'] = df_raw['TORQUE']
    df['Deflection [deg]'] = df_raw['POSITION'] * 360
    df['Desired Torque [Nm]'] = df_raw['CONTROL_TORQUE']
    df['Q_current/5'] = df_raw['Q_CURRENT']/5
    df['test_nr'] = df_raw['test_nr']

    # print(f'Sample rate: {1/(df["Time [s]"].diff().mean())} Hz')
    df['Time [s]'] = pd.to_datetime(df['Time [s]'], unit='s')
    df.set_index('Time [s]', inplace=True)


    print(f"Test output orientation: {df['Deflection [deg]'].mean()} deg")
    df['Deflection [deg]'] -= df['Deflection [deg]'].mean()

    
    TEST_NR_OFFSET = 100
    play_tests = df[df['test_nr'] < TEST_NR_OFFSET].copy()
    stiffness_tests = df[df['test_nr'] >= TEST_NR_OFFSET].copy()
    stiffness_tests['test_nr'] = stiffness_tests['test_nr'] - TEST_NR_OFFSET

    play_test_mean = average_test_repetitions(play_tests, 'test_nr')
    stiffness_test_mean = average_test_repetitions(stiffness_tests, 'test_nr')


    return play_test_mean, stiffness_test_mean

In [4]:


def load_speeddata(filename):
    print(f'prepping {filename}')
    df_raw = pd.read_csv(filename)

    df = pd.DataFrame()
    df['Time [s]'] = df_raw['TIME']
    df['Motor Speed [rpm]'] = df_raw['VELOCITY']
    df['Motor Torque [Nm]'] = df_raw['TORQUE']

    df['Time [s]'] = pd.to_datetime(df['Time [s]'], unit='s')
    df.set_index('Time [s]', inplace=True)

    return df

In [5]:

if __name__ == '__main__':
    for test_name, (torque_files, speed_file) in test_data.items():
        print(f'Processing {test_name}')

        fig, axs = plt.subplots(1, 2, figsize=(13, 6))
        colors = plt.cm.viridis(np.linspace(0, 1, len(torque_files)))
        for i, torque_file in enumerate(torque_files):
            play_test, stiffness_test = load_torquedata(torque_file)

            ############### analyse play and stiffness ###############
            def get_rising_slopes(df):
                # split data in to positive and negative slopes. remove the hysteresis part
                max_torque_idx = df['Desired Torque [Nm]'].idxmax()
                pos_slope = df.loc[:max_torque_idx]

                # find zero crossing of desired torque
                min_torque_idx = df['Desired Torque [Nm]'].idxmin()
                zero_torque_idx = df['Desired Torque [Nm]'][max_torque_idx:min_torque_idx].abs().idxmin()
                neg_slope = df.loc[zero_torque_idx:min_torque_idx]

                return pos_slope, neg_slope


            # estimate play
            play_pos_slope, play_neg_slope = get_rising_slopes(play_test)
            pos_play = play_pos_slope['Deflection [deg]'].iloc[0]
            neg_play = play_neg_slope['Deflection [deg]'].iloc[0]
            print(f'Play: {neg_play-pos_play} deg ({pos_play} pos, {neg_play} neg)')


            # estimate stiffness
            stiffness_pos_slope, stiffness_neg_slope = get_rising_slopes(stiffness_test)
            stiffness_pos_slope = stiffness_pos_slope[stiffness_pos_slope['Deflection [deg]'] > neg_play]
            stiffness_neg_slope = stiffness_neg_slope[stiffness_neg_slope['Deflection [deg]'] < pos_play]

            # do polyfit
            pos_stifness, pos_offset = np.polyfit(stiffness_pos_slope['Motor Torque [Nm]'], stiffness_pos_slope['Deflection [deg]'], 1)
            neg_stifness, neg_offset  = np.polyfit(stiffness_neg_slope['Motor Torque [Nm]'], stiffness_neg_slope['Deflection [deg]'], 1)

            print(f'Positive stiffness: {pos_stifness} deg/Nm, offset: {pos_offset} deg')
            print(f'Negative stiffness: {neg_stifness} deg/Nm, offset: {neg_offset} deg')

            ##################################

            axs[0].plot(play_test['Motor Torque [Nm]'], play_test['Deflection [deg]'], label=f'play test {i}', color=colors[i])
            axs[1].plot(stiffness_test['Motor Torque [Nm]'], stiffness_test['Deflection [deg]'], label=f'stiffness test {i}', color=colors[i])

        for ax in axs:
            ax.legend()
            ax.set_xlabel('Motor Torque [Nm]')
            ax.set_ylabel('Deflection [deg]')


        speed_data = load_speeddata(speed_file)

        fig, axs = plt.subplots(1, 1, figsize=(10, 10))
        axs.plot(speed_data['Motor Speed [rpm]'], speed_data['Motor Torque [Nm]'])



        plt.show()
        print('done')

NameError: name 'test_data' is not defined